In [2]:
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
from topoPy import *

df = nc.Dataset('../data/icon_grid_0012_R02B04_G_linked.nc')

clat = df.variables['clat'][:]
clon = df.variables['clon'][:]
clat_verts = df.variables['clat_vertices'][:]
clon_verts = df.variables['clon_vertices'][:]
links = df.variables['links'][:]

# clat = clat*(180/np.pi)
# clon = clon*(180/np.pi)
# clat_verts = clat_verts*(180/np.pi)
# clon_verts = clon_verts*(180/np.pi)

datfile = '../data/GMTED2010_topoGlobal_SGS_30ArcSec.nc'
var = {'name':'topo','units':'m'}

np.random.seed(555)
# icon_cell_indexes = np.sort([440, 19442, 5595, 5026, 4793, 4631])
icon_cell_indexes = np.random.randint(0,np.size(clat)-1,36)

# icon_cell_indexes = [3027,3028,3029]
# Mount Ebrus; Firehorn; Taunus; Pirin; Langtang; ???
print(icon_cell_indexes)
print(icon_cell_indexes.shape)

[ 4782 15337  7549  6694  9622   233  5616 16992 17492  8086  3658 10709
  5147 14394 19184 12993 12794 19635  1613 15737 15939  7443  5304 16371
  2308 10345   780  8212 13981 13600  3441 12496  4431 17984 17842 13785]
(36,)


In [3]:
comp_clat = clat[icon_cell_indexes]
comp_clon = clon[icon_cell_indexes]
comp_clat_verts = clat_verts[icon_cell_indexes]
comp_clon_verts = clon_verts[icon_cell_indexes]

ncfile = Dataset('../data/icon_compact.nc',mode='w') 
print(ncfile)

cell = ncfile.createDimension('cell', np.size(comp_clat))     # latitude axis
nv = ncfile.createDimension('nv', 3)    # longitude axis
for dim in ncfile.dimensions.items():
    print(dim)

ncfile.title='Compact ICON grid for testing and debugging purposes'
print(ncfile.title)

clat = ncfile.createVariable('clat', np.float32, ('cell',))
clat.units = 'radian'
clat.long_name = 'center latitude'

clon = ncfile.createVariable('clon', np.float32, ('cell',))
clon.units = 'radian'
clon.long_name = 'center longitude'

clat_verts = ncfile.createVariable('clat_vertices', np.float32, ('cell','nv',))
clat_verts.units = 'radian'

clon_verts = ncfile.createVariable('clon_vertices', np.float32, ('cell','nv',))
clon_verts.units = 'radian'

clat[:] = comp_clat
clon[:] = comp_clon
clat_verts[:,:] = comp_clat_verts
clon_verts[:,:] = comp_clon_verts

print(clon_verts[:,:])

ncfile.close(); print('Dataset is closed!')

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): 
    variables(dimensions): 
    groups: 
('cell', <class 'netCDF4._netCDF4.Dimension'>: name = 'cell', size = 36)
('nv', <class 'netCDF4._netCDF4.Dimension'>: name = 'nv', size = 3)
Compact ICON grid for testing and debugging purposes
[[-0.06764244 -0.09624738 -0.03211919]
 [ 1.081351    1.0660676   1.1036539 ]
 [-2.1275022  -2.1011112  -2.1402485 ]
 [ 2.5532556   2.533109    2.5727484 ]
 [ 0.19529866  0.21523526  0.17626585]
 [ 1.6172599   1.5462693   1.6375297 ]
 [ 1.336554    1.3575194   1.317224  ]
 [ 2.8069417   2.864542    2.7888591 ]
 [-1.2566371  -1.2566371  -1.329176  ]
 [-2.9417224  -2.9224007  -2.9613843 ]
 [-1.0143936  -0.97144777 -1.0336092 ]
 [ 1.5775877   1.551702    1.5920645 ]
 [ 1.3577651   1.337822    1.3777082 ]
 [ 0.70950335  0.68921053  0.72944653]
 [-0.23339023 -0.24578841 -0.18092659]
 [-2.0215986  -2.002043   -2.0410075 ]
 [-1.9667751  -1.9888427  -1.94

In [4]:
links_tmp = links[icon_cell_indexes].flatten()
links_tmp = links_tmp[np.where(links_tmp > 0)]
links_tmp = np.sort(list(set(links_tmp)))
links_tmp -= 1

print(links_tmp)

lon, lat, z = readnc(datfile, var)
nrecords = np.shape(z)[0]; nlon = np.shape(lon)[1]; nlat = np.shape(lat)[1]

sz_tmp = np.size(links_tmp)
# print(nlat,nlon, np.size(links_tmp))

compactified_topo = np.zeros((sz_tmp,nlat, nlon))
print(compactified_topo.shape)

compactified_lat = np.zeros((sz_tmp,nlat))
compactified_lon = np.zeros((sz_tmp,nlon))
    
for i,lnk in enumerate(links_tmp):
    print("i, lnk = ", (i, lnk))
    compactified_lat[i] = lat[lnk]
    compactified_lon[i] = lon[lnk]
    compactified_topo[i] = z[lnk]
    
del lat, lon, z

ncfile = Dataset('../data/topo_compact.nc',mode='w',format='NETCDF4_CLASSIC') 
print(ncfile)

nfiles = ncfile.createDimension('nfiles', sz_tmp)
lat = ncfile.createDimension('lat', nlat)
lon = ncfile.createDimension('lon', nlon)
for dim in ncfile.dimensions.items():
    print(dim)

ncfile.title='Compact GMTED2010 USGS Topography grid for testing and debugging purposes'
print(ncfile.title)

lat = ncfile.createVariable('lat', np.float32, ('nfiles','lat'))
lat.units = 'degrees'

lon = ncfile.createVariable('lon', np.float32, ('nfiles','lon'))
lon.units = 'degress'

topo = ncfile.createVariable('topo', np.float32, ('nfiles','lat','lon'))
topo.units = 'meters'

lat[:,:] = compactified_lat
lon[:,:] = compactified_lon
topo[:,:,:] = compactified_topo

ncfile.close(); print('Dataset is closed!')

[ 0  2  6 12 14 15 17 19 21 22 27 31 32 33 35 36 37 39 41 43 48 50 51 54
 55 56 57 58 59 60 61 62 63 64 65 66 67 69 70 76 77 84 85 92 97]
{'lat': <class 'netCDF4._netCDF4.Variable'>
float32 lat(nfiles, lat)
    units: degrees
unlimited dimensions: 
current shape = (108, 2400)
filling on, default _FillValue of 9.969209968386869e+36 used, 'lon': <class 'netCDF4._netCDF4.Variable'>
float32 lon(nfiles, lon)
    units: degrees
unlimited dimensions: 
current shape = (108, 3600)
filling on, default _FillValue of 9.969209968386869e+36 used, 'topo': <class 'netCDF4._netCDF4.Variable'>
float32 topo(nfiles, lat, lon)
    units: meters
unlimited dimensions: 
current shape = (108, 2400, 3600)
filling on, default _FillValue of 9.969209968386869e+36 used}
(45, 2400, 3600)
i, lnk =  (0, 0)
i, lnk =  (1, 2)
i, lnk =  (2, 6)
i, lnk =  (3, 12)
i, lnk =  (4, 14)
i, lnk =  (5, 15)
i, lnk =  (6, 17)
i, lnk =  (7, 19)
i, lnk =  (8, 21)
i, lnk =  (9, 22)
i, lnk =  (10, 27)
i, lnk =  (11, 31)
i, lnk =  (12, 32

In [ ]:
lon, lat, z = readnc(datfile, var)
nrecords = np.shape(z)[0]; nlon = np.shape(lon)[1]; nlat = np.shape(lat)[1]

In [19]:
print(lon[1][:])
print(lon[2][:])
print(lon[3][:])
print(lon[4][:])

[4.1666669e-03 1.2500000e-02 2.0833334e-02 ... 2.9979166e+01 2.9987499e+01
 2.9995832e+01]
[30.004168 30.0125   30.020834 ... 59.979168 59.9875   59.995834]
[-2.9995832e+01 -2.9987499e+01 -2.9979166e+01 ... -2.0833334e-02
 -1.2500000e-02 -4.1666669e-03]
[30.004168 30.0125   30.020834 ... 59.979168 59.9875   59.995834]


In [20]:
print(lat[1][:])
print(lat[2][:])
print(lat[3][:])
print(lat[4][:])

[-9.995833 -9.9875   -9.979167 ...  9.979167  9.9875    9.995833]
[10.004167 10.0125   10.020833 ... 29.979166 29.9875   29.995832]
[10.004167 10.0125   10.020833 ... 29.979166 29.9875   29.995832]
[-9.995833 -9.9875   -9.979167 ...  9.979167  9.9875    9.995833]
